## Script Python per la creazione delle tabelle per anno

### Caricamento delle librerie

In [1]:
#librerie generiche python
import pandas as pd
import numpy as np

#librerie per caricamento file sdg
import os
from os import listdir
from os.path import join

#librerie per operazioni su tabella
from sklearn.preprocessing import MinMaxScaler

pd.set_option('max_columns', 45)

### Lettura dei file all'interno della cartella \data_sdg\

In [2]:
#lettura cartelle all'interno della cartella data
path = os.getcwd() + "\\data_sdg\\"
sdg_folder_name = [f for f in listdir(path)]

#lettura dei file all'interno delle cartelle
sdg_table_files = dict()
for f in sdg_folder_name:
    folder_path = join(path,f)
    sdg_table_files[f] = [join(folder_path,t) for t in listdir(folder_path)]

In [3]:
#definizione dei nomi degli attributi scelti per gli sdg
sdg_1 = ['dwelling_condition','home_warm','poverty']
sdg_4 = ['drop_rate','edu_0_2_18_24','edu_3_4_18_24','edu_5_8_18_24',
         'edu_0_2_25_34','edu_3_4_25_34','edu_5_8_25_34',
         'edu_pre_scuola']
sdg_8 = ['real_gdp_pro_capite','dmc','unemployment_rate',
         'young_people_not_employed','killed_at_work']
sdg_9 = ['ghg_emissions','rd_expenditure','mobile_subs','fixed_telephone_subs',
         'fixed_broadband_subs', 'internet_use']

final_col_name = ['dmc', 'drop_rate', 'dwelling_condition', 'edu_0_2_18_24', 'edu_0_2_25_34', 'edu_3_4_18_24',
                  'edu_3_4_25_34', 'edu_5_8_18_24', 'edu_5_8_25_34', 'edu_pre_scuola', 'fixed_broadband_subs', 
                  'fixed_telephone_subs', 'ghg_emissions', 'home_warm', 'internet_use', 'killed_at_work', 
                  'mobile_subs', 'poverty', 'rd_expenditure', 'real_gdp_pro_capite', 'unemployment_rate', 
                  'young_people_not_employed']

#definizione delle colonne da tenere delle tabelle di partenza
anni = ['country','2010','2011','2012','2013','2014','2015','2016']

#definizione dei paesi per cui si vogliono i dati
paesi = ['Austria','Belgium','Bulgaria','Cyprus','Czechia','Denmark','EU (28 countries)','Estonia',
         'Finland','France','Germany','Greece','Hungary','Ireland','Italy',
         'Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland',
         'Portugal','Romania','Slovakia','Slovenia','Spain','Sweden','United Kingdom']


In [4]:
#lettura delle tabelle con i dati per calcolare gli sdg
sdg_tables = {}
for sdg in sdg_table_files.keys():
    tables = list()
    for table in sdg_table_files[sdg]:
        tmp = pd.read_excel(table, sheet_name = None)
        tables.append(tmp)
    sdg_tables[sdg] = tables 

### Eliminazione dalle tabelle di partenza degli headers e dei footers

In [39]:
'''
Trovare inizio e fine tabella (da geo a : o nan)
-
'''

def get_row_table_begin(table):
    for index,row in table.iterrows():
        if ('geo' in str(row[table.columns[0]]).lower()) or ('afghanistan' in str(row[table.columns[0]]).lower()):
            return(index)
        
def get_row_table_end(table):
    start = get_row_table_begin(table)
    for index,row in table.iterrows():
        if index > start:
            if (':' in str(row[table.columns[0]]).lower()) or (pd.isnull(row[table.columns[0]])):
                return start,index
            elif (index == len(table.index)-1):
                return (start -1),index
'''
Estrarre nomi colonne da slice tabella
'''

def get_new_columns(table):
    new_col = ['country']
    for year in table.iloc[0,:].values[1:]:
        new_col.append(str(year).split('.')[0])
    return new_col

In [ ]:
'''
Creazione nuovo dizionario con le tabelle originali tagliate
'''
sdg_tables_cropped = {}
for sdg in sdg_tables:
    tables = list()
    for table in sdg_tables[sdg]:
        for k in table.keys():
            if table[k].index[1] == 'Afghanistan':
                table_ = table[k].reset_index()
                ini, fin = get_row_table_end(table_)
                new_col = get_new_columns(table_.iloc[ini:fin,:])
                tmp = table_.iloc[ini:fin,:]
                tables.append({ k: pd.DataFrame(tmp.iloc[1:,:].values, columns = new_col)})
            else:
                ini, fin = get_row_table_end(table[k])
                new_col = get_new_columns(table[k].iloc[ini:fin,:])
                tmp = table[k].iloc[ini:fin,:]
                tables.append({ k: pd.DataFrame(tmp.iloc[1:,:].values, columns = new_col)})
    sdg_tables_cropped[sdg] = tables        

In [40]:
'''
Eliminazione colonne degli anni che non interessano e delle righe dei paesi non considerati
'''
country_to_rename = {'European Union - 28 countries' : 'EU (28 countries)',
                     'Germany (until 1990 former territory of the FRG)' : 'Germany',
                     'Czech Republic' : 'Czechia'}

for sdg in sdg_tables_cropped:
    for table in sdg_tables_cropped[sdg]:
        for k in table.keys():
            tmp = table[k][anni] #subsetting delle colonne non utilizzate
            tmp = tmp.sort_values('country').reset_index(drop = True)
            for index,row in tmp.iterrows(): #correzione dei nomi in country 
                if(row['country'] in country_to_rename):
                    tmp.loc[index,'country'] = country_to_rename[row['country']]
            tmp = tmp[tmp.country.isin(paesi)]
            if 'EU (28 countries)' in list(tmp['country']):
                table[k] = tmp.sort_values('country').reset_index(drop = True)
            else:
                tmp = tmp.append(pd.DataFrame([['EU (28 countries)',':',':',':',':',':',':',':']], columns = anni))
                tmp = tmp.sort_values('country').reset_index(drop = True)
                table[k] = tmp


### Definizione delle funzioni di pre-processing

In [44]:
'''
Operazioni sulle colonne da effettuare

    - eliminazione : e sostituzione con NaN
    - x/100 
    - /1000000 e arrotondato a 6 cifre
    - x - 1
    - 1 - x 

'''

def prep_1(value): return str(value).replace(':', '')

def prep_2(value): 
    try:
        if value != '': 
            return float(value)/100
        else:
            return np.nan
    except:
        print(value)

def prep_3(value): return round(float(value)/1000000,6)

def prep_4(value): return (value - 1) 

def prep_5(value): return (1 - value)


Il dizionario qui sotto definito servirà per applicare ad ogni indicatore tutte le funzioni di pre-processing necessarie all'interno di un ciclo, senza doverle applicare una alla volta

In [45]:
col_func_apply = {'dmc': [prep_1, prep_2], #da normalizzare
                  'drop_rate' : [prep_1, prep_2], 
                  'dwelling_condition' : [prep_1, prep_2, prep_5], 
                  'edu_0_2_18_24' : [prep_1, prep_2], 
                  'edu_0_2_25_34': [prep_1, prep_2], 
                  'edu_3_4_18_24': [prep_1, prep_2], 
                  'edu_3_4_25_34': [prep_1, prep_2], 
                  'edu_5_8_18_24': [prep_1, prep_2], 
                  'edu_5_8_25_34': [prep_1, prep_2], 
                  'edu_pre_scuola': [prep_1, prep_2], 
                  'fixed_broadband_subs': [prep_1, prep_2], 
                  'fixed_telephone_subs': [prep_1, prep_2],
                  'ghg_emissions': [prep_1],   # da normalizzare 
                  'home_warm': [prep_1, prep_2, prep_5], 
                  'internet_use': [prep_1, prep_2], 
                  'killed_at_work': [prep_1, prep_3, prep_5], 
                  'mobile_subs': [prep_1, prep_2, prep_4], 
                  'poverty': [prep_1, prep_2, prep_5], 
                  'rd_expenditure': [prep_1, prep_2], 
                  'real_gdp_pro_capite': [prep_1], #da normalizzare
                  'unemployment_rate': [prep_1, prep_2, prep_5], 
                  'young_people_not_employed': [prep_1, prep_2, prep_5],
                  'population' : [prep_1]}


### Applicazione delle funzioni di pre-processing

In [46]:
for sdg in sdg_tables_cropped:
    for table in sdg_tables_cropped[sdg]:
        for k in table.keys():
            for anno in anni[1:]:
                for f in col_func_apply[k]:
                    table[k][anno] = table[k][anno].apply(f)

In [48]:
#normalizzazione colonne
to_norm = ['dmc','ghg_emissions','real_gdp_pro_capite']
for sdg in sdg_tables_cropped:
    for table in sdg_tables_cropped[sdg]:
        for k in table.keys():
            if (k in to_norm):
                for col in table[k].columns[1:]:
                    min_max_scaler = MinMaxScaler()
                    scaled = min_max_scaler.fit_transform(table[k][col].values.astype(float).reshape(-1, 1))
                    table[k].loc[:,col] = scaled
                    if k == 'ghg_emissions':
                        table[k].loc[:,col] = table[k][col].apply(lambda x: 1 - x)
                  

### Creazione delle tabelle per anno

In [49]:
sdg_per_anni = {}
for anno in anni[1:]:
    sdg_anno = pd.DataFrame(data = paesi, columns = ['country'])
    for sdg in sdg_tables_cropped:
        for table in sdg_tables_cropped[sdg]:
            for k in table.keys():
                sdg_anno[k] = table[k][anno].astype(float)
        sdg_per_anni[anno] = sdg_anno

### Calcolo dei missing value in EU 28 con la media pesata

In [50]:
for anno in sdg_per_anni.keys():
    population = sdg_per_anni[anno][sdg_per_anni[anno]['country'] != 'EU (28 countries)']['population']
    sum_p = population.sum()
    for col in sdg_per_anni[anno].columns:
        if pd.isna(sdg_per_anni[anno].loc[6,col]):
            value = sdg_per_anni[anno][sdg_per_anni[anno]['country'] != 'EU (28 countries)'][col]
            media_eu = (value * population).sum() / sum_p
            sdg_per_anni[anno].loc[6,col] = round(media_eu,4)

### Scrittura delle tabelle per anno

In [30]:
save_path = os.getcwd() + "\\final_table\\"
os.mkdir(save_path)
for anno in sdg_per_anni.keys():
    table_name = "data_" + anno + ".csv"
    sdg_per_anni[anno].to_csv(join(save_path,table_name), encoding='utf-8', sep = ',', header = True, index = 'False')